In [1]:
import pandas as pd
import sqlite3
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns

conn = sqlite3.connect("./data/output/home_ahead.db")
cur = conn.cursor()

In [2]:
ahead_df = pd.read_sql_query("select * from processed_data;", conn)

In [3]:
ahead_df.head()

,GAME_ID,total_events,home_ahead,pct_home_ahead,period_1_events,period_2_events,period_3_events,period_4_events,period_5_events,period_6_events,...,period_1_ahead,period_2_ahead,period_3_ahead,period_4_ahead,period_5_ahead,period_6_ahead,period_7_ahead,home_win,period_8_events,period_8_ahead
0,20001149,167,15,0.089820,33.0,37.0,43.0,35.0,19.0,NaN,...,8.0,0.0,0.0,7.0,0.0,NaN,NaN,0,NaN,NaN
1,20000923,161,78,0.484472,28.0,37.0,31.0,23.0,15.0,13.0,...,7.0,26.0,25.0,0.0,5.0,2.0,13.0,1,NaN,NaN
2,20000936,155,109,0.703226,29.0,40.0,35.0,36.0,15.0,NaN,...,3.0,38.0,35.0,32.0,1.0,NaN,NaN,0,NaN,NaN
3,20000432,155,155,1.000000,32.0,39.0,35.0,49.0,NaN,NaN,...,32.0,39.0,35.0,49.0,NaN,NaN,NaN,1,NaN,NaN
4,20001024,151,30,0.198675,30.0,34.0,32.0,41.0,14.0,NaN,...,10.0,0.0,0.0,10.0,10.0,NaN,NaN,1,NaN,NaN


In [4]:
len(ahead_df)

20505

In [5]:
ahead_df['GAME_ID'].unique()

array([20001149, 20000923, 20000936, ..., 21700283, 21700079, 21701058])

In [16]:
def year_parse(row):
    '''returns year for the start of the season based on the game id,
       since we know these are all year 2000 and beyond we hardcode the prefix
    '''
    game_id = row['GAME_ID']
    year = "20{}".format(str(game_id)[1:3])
    return year

In [17]:
ahead_df['year'] = ahead_df.apply(lambda x: year_parse(x), axis=1)

In [18]:
ahead_df.head()

,GAME_ID,total_events,home_ahead,pct_home_ahead,period_1_events,period_2_events,period_3_events,period_4_events,period_5_events,period_6_events,...,period_2_ahead,period_3_ahead,period_4_ahead,period_5_ahead,period_6_ahead,period_7_ahead,home_win,period_8_events,period_8_ahead,year
0,20001149,167,15,0.089820,33.0,37.0,43.0,35.0,19.0,NaN,...,0.0,0.0,7.0,0.0,NaN,NaN,0,NaN,NaN,2000
1,20000923,161,78,0.484472,28.0,37.0,31.0,23.0,15.0,13.0,...,26.0,25.0,0.0,5.0,2.0,13.0,1,NaN,NaN,2000
2,20000936,155,109,0.703226,29.0,40.0,35.0,36.0,15.0,NaN,...,38.0,35.0,32.0,1.0,NaN,NaN,0,NaN,NaN,2000
3,20000432,155,155,1.000000,32.0,39.0,35.0,49.0,NaN,NaN,...,39.0,35.0,49.0,NaN,NaN,NaN,1,NaN,NaN,2000
4,20001024,151,30,0.198675,30.0,34.0,32.0,41.0,14.0,NaN,...,0.0,0.0,10.0,10.0,NaN,NaN,1,NaN,NaN,2000


Lets first just make sure we know the difference between games that ended in 4 periods and those that went into overtime.

In [19]:
ot_games = ahead_df.dropna(subset=['period_5_events'])

In [20]:
len(ot_games)

1283

A small fraction ((1283/20505)*100) = 6.26% of games went into overtime, let's ignore these

In [22]:
ot_games.head()

,GAME_ID,total_events,home_ahead,pct_home_ahead,period_1_events,period_2_events,period_3_events,period_4_events,period_5_events,period_6_events,...,period_2_ahead,period_3_ahead,period_4_ahead,period_5_ahead,period_6_ahead,period_7_ahead,home_win,period_8_events,period_8_ahead,year
0,20001149,167,15,0.089820,33.0,37.0,43.0,35.0,19.0,NaN,...,0.0,0.0,7.0,0.0,NaN,NaN,0,NaN,NaN,2000
1,20000923,161,78,0.484472,28.0,37.0,31.0,23.0,15.0,13.0,...,26.0,25.0,0.0,5.0,2.0,13.0,1,NaN,NaN,2000
2,20000936,155,109,0.703226,29.0,40.0,35.0,36.0,15.0,NaN,...,38.0,35.0,32.0,1.0,NaN,NaN,0,NaN,NaN,2000
4,20001024,151,30,0.198675,30.0,34.0,32.0,41.0,14.0,NaN,...,0.0,0.0,10.0,10.0,NaN,NaN,1,NaN,NaN,2000
5,20000847,150,68,0.453333,32.0,30.0,29.0,23.0,17.0,19.0,...,0.0,8.0,20.0,2.0,9.0,NaN,1,NaN,NaN,2000


In [23]:
no_ot_df = ahead_df[ahead_df['period_5_events'].isna()]

In [24]:
len(no_ot_df)

19222

In [37]:
len(no_ot_df[(no_ot_df.period_4_ahead < 4) & (no_ot_df.home_win == 1)].home_win)

84

In [40]:
len(no_ot_df[(no_ot_df.period_4_ahead < 10) & (no_ot_df.home_win == 1)].home_win)

429

In [44]:
no_ot_df.describe()

,GAME_ID,total_events,home_ahead,pct_home_ahead,period_1_events,period_2_events,period_3_events,period_4_events,period_5_events,period_6_events,...,period_1_ahead,period_2_ahead,period_3_ahead,period_4_ahead,period_5_ahead,period_6_ahead,period_7_ahead,home_win,period_8_events,period_8_ahead
count,1.922200e+04,19222.000000,19222.000000,19222.000000,19222.000000,19222.000000,19222.000000,19222.000000,0.0,0.0,...,19222.000000,19222.000000,19222.000000,19222.000000,0.0,0.0,0.0,19222.000000,0.0,0.0
mean,2.080965e+07,116.864322,64.354073,0.550088,27.810374,29.637343,29.257465,30.159141,NaN,NaN,...,13.970190,16.164863,16.601862,17.617157,NaN,NaN,NaN,0.601706,NaN,NaN
std,5.005453e+05,11.850622,41.281502,0.346082,4.729705,5.057316,5.024050,5.807475,NaN,NaN,...,10.362429,12.873645,13.224255,13.876539,NaN,NaN,NaN,0.489559,NaN,NaN
min,2.000000e+07,77.000000,0.000000,0.000000,11.000000,11.000000,12.000000,13.000000,NaN,NaN,...,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,0.000000,NaN,NaN
25%,2.040040e+07,109.000000,25.000000,0.219298,25.000000,26.000000,26.000000,26.000000,NaN,NaN,...,4.000000,2.000000,1.000000,0.000000,NaN,NaN,NaN,0.000000,NaN,NaN
50%,2.080060e+07,116.000000,69.000000,0.596837,28.000000,29.000000,29.000000,30.000000,NaN,NaN,...,14.000000,18.000000,19.000000,22.000000,NaN,NaN,NaN,1.000000,NaN,NaN
75%,2.120104e+07,124.000000,101.000000,0.887188,31.000000,33.000000,32.000000,34.000000,NaN,NaN,...,23.000000,28.000000,28.000000,29.000000,NaN,NaN,NaN,1.000000,NaN,NaN
max,2.170123e+07,173.000000,171.000000,1.000000,48.000000,51.000000,52.000000,61.000000,NaN,NaN,...,47.000000,51.000000,52.000000,61.000000,NaN,NaN,NaN,1.000000,NaN,NaN


In [56]:
p4_win_bins = pd.cut(no_ot_df[no_ot_df.home_win==1]['period_4_ahead'], [0, 10, 20, 30, 40, 50, 60, 70])

In [59]:
p4_loss_bins = pd.cut(no_ot_df[no_ot_df.home_win==0]['period_4_ahead'], [0, 10, 20, 30, 40, 50, 60, 70])

In [57]:
no_ot_df[no_ot_df.home_win==1].groupby(p4_win_bins)['period_4_ahead'].agg(['count'])

,count
period_4_ahead,
"(0, 10]",505
"(10, 20]",1405
"(20, 30]",5727
"(30, 40]",3580
"(40, 50]",333
"(50, 60]",15
"(60, 70]",1


In [60]:
no_ot_df[no_ot_df.home_win==0].groupby(p4_loss_bins)['period_4_ahead'].agg(['count'])

,count
period_4_ahead,
"(0, 10]",1633
"(10, 20]",658
"(20, 30]",274
"(30, 40]",20
"(40, 50]",1
"(50, 60]",0
"(60, 70]",0
